In [2]:
import pandas as pd
import numpy as np
import os
#from envs.portfolio_env import BinancePortfolioEnv


path = "/Users/yusuf.bektas/Quant/drl-crypto/data/features/600s"
X = []
keys = []
for file in os.listdir(path):
    if file.endswith(".parquet"):
        df = pd.read_parquet(os.path.join(path, file))
        df = df.dropna().drop(columns=["price_last","buy_qty","sell_qty"])
        df = df.set_index("ts")
        X.append(df)
        keys.append(file.split("-")[0])
X = pd.concat(X, axis=1,keys=keys).sort_index().ffill().replace([np.inf,-np.inf],np.nan).dropna()
bids = X.xs("best_bid", level=1, axis=1)
asks = X.xs("best_ask", level=1, axis=1)
X = X.drop(["best_bid","best_ask"], axis=1, level=1)
print("X ")
print(X.info())

print("Asks")
print(asks.info())

print("Bids")
print(bids.info())




KeyboardInterrupt: 

In [ ]:
# current MultiIndex:  level-0 = symbol, level-1 = feature name
symbols   = bids.columns.tolist()          # ['BNBUSDT', 'XRPUSDT', ...]  (A = 7)
features  = X.columns.levels[1].tolist()   # 15 fe>ature names              (F = 15)

T, A, F = len(X), len(symbols), len(features)
cube = np.empty((T, A, F), dtype=np.float32)

for a, sym in enumerate(symbols):
    cube[:, a, :] = X[sym].values          # (T, F)


from drl_crypto.envs.portfolio_env import BinancePortfolioEnv   # use package name

env = BinancePortfolioEnv(
    feature_cube=cube,
    bids=bids.values.astype(np.float32),    # (T, A)
    asks=asks.values.astype(np.float32),    # (T, A)
    window=50
)

obs, _ = env.reset(seed=0)
print("obs OK?", env.observation_space.contains(obs))   # → True

act = env.action_space.sample()
obs, r, term, trunc, _ = env.step(act)
print("reward:", r, "terminated?", term)                 # finite reward, False


ValueError: could not broadcast input array from shape (2438155,15) into shape (2438155,17)